# Skeleton of Assignment 4:
    test if the distribution of 
    
    1) trip duration of bikers that ride during the day vs night
    
    2) age of bikers for trips originating in Manhattan and in Brooklyn
    
    are different. Use 3 tests: KS, Pearson's, Spearman's. 
    
    Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 
    
    For the KS do the test with the entire dataset and with a subset 200 times smaller
    
    Choose a single significant threshold for the whole exercise. 
    
    For each test phrase the Null Hypothesis in words.
    
    Describe the return of the scipy function you use in each case.
    
    State the result in terms of rejection of the Null.

In [1]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np

%pylab inline
import os

Populating the interactive namespace from numpy and matplotlib


# Read in data
I am reading in data from January 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [184]:
df1 = pd.read_csv('https://s3.amazonaws.com/tripdata/201611-citibike-tripdata.zip', compression = 'zip')
df2 = pd.read_csv('https://s3.amazonaws.com/tripdata/201612-citibike-tripdata.zip', compression = 'zip')

In [186]:
df = pd.concat([df1, df2])

# 1) trip duration of bikers that ride during the day vs night

**H0: there is no statistical difference in the trip duration of bikers at day time vs at night**
$$ \alpha = 0.05 $$

In [187]:
df1 = df[['Trip Duration', 'Start Time']]
df1['date'] = pd.to_datetime(df1['Start Time'])
df1['hour'] = list(map(lambda x: x.hour, df1.date))

/home/baiyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/baiyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [188]:
df1['day_or_night'] = list(map(lambda x: int((x >= 6) & (x <= 19)), df1.hour))

/home/baiyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [189]:
df1['day'] = df1.day_or_night
df1['night'] = df1.day_or_night == 0
df1.night = df1.night.astype(int)
df1 = df1.dropna(subset = ['Trip Duration'])

/home/baiyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/baiyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/baiyue/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

lets run the scipy KS test

In [190]:
import scipy.stats
#remember that your imports should all be at the top. I leave it here to hightlight that this package is needed at this point of the workflow

# KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

the KS test in scipy returns the p-value BUT make sure you understand what the NULL is! read the documentation carefully! what is the null hypothesis that you can/cannot reject?

In [191]:
ks = scipy.stats.ks_2samp(df1[df1.day == 1]['Trip Duration'], df1[df1.day == 0]['Trip Duration'])
print (ks)

Ks_2sampResult(statistic=0.066039615940545859, pvalue=0.0)


**  FILL IN THE CELL BELOW!**

###### since the pvalue is lower than 0.05, we reject the null hypothesis, which implies that the data set of trips took place at night and at day time are statistically different

The scipy.stats KS test already tells me the significance and the p-value. 

The next few cells are here just to show you how you would obtain the same result by hand, but they are **not required**. 

Remember: the Null hypothesis is rejected if 

$D_KS(n1,n2) > c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$

(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!! 

http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/

But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.

# Now retest using a test for correlation. 

That will answer a slightly different question though - formulate the NULL appropriately. The tests for correlations (generally) requires the variable to be paired, so that I can tell if x changes does y change similarly. But the datasets are of different size! You will need to reduce them to the same size. You can do that by subsampling of the data: take only 1 ride every of 200, which you can achieve "slicing and broadcasting" the array or using one of the python function (built in python numpy.random.choice() functions for example: Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

        .. versionadded:: 1.7.0

Parameters
...

But make sure you understand how to use it! there is an option "replace" which you should think about.

# Pearson's  test for correlation

** notice that the Pearson's is a pairwise test: the samples need to be **
 a. the same size
 b. sorted! (how??)
    
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr



In [195]:
# your code here!
# wrangle the data as needed
# please perform the Pearson's test 
# and tell me what you find in terms of NULL hypothesis
r1, pval1 = scipy.stats.pearsonr(np.sort(df1['Trip Duration']*df1.day), np.sort(df1['Trip Duration']*df1.night))
print(r1, pval1)
print('the correlation between the trip duration taking place at day time and night time is strong and statistically significant, hence fail reject null hypothesis ')

0.964954651147 0.0
the correlation between the trip duration taking place at day time and night time is strong and statistically significant, hence fail reject null hypothesis 


# Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

In [193]:
# your code here!
# wrangle the data as needed
# please perform the Spearman's test and tell me what you find in terms of NULL hypothesisc
r11, pval11= scipy.stats.spearmanr(df1['Trip Duration']*df1.day, df1['Trip Duration']*df1.night)
print(r11, pval11)
print('The correlation between the two data set is weak (the two data set are statistically different) and we reject the null hypothesis since p < alpha')

-0.577850103554 0.0
The correlation between the two data set is weak (the two data set are statistically different) and we reject the null hypothesis since p < alpha


In [172]:
df.location = list(zip(df['Start Station Latitude'],df['Start Station Longitude']))

In [173]:
bk_zip = [11212, 11213, 11216, 11233, 11238, 11209, 11214, 11228,1204,
          11218, 11219, 11230,11234, 11236, 11239,11223, 11224, 11229, 
          11235,11201, 11205, 11215, 11217, 11231,1203, 11210, 11225, 
          11226,11207, 11208,11211, 11222,11220, 11232,1206, 11221, 11237]
          
mn_zip = [10026, 10027, 10030, 10037, 10039,10001, 10011, 10018, 
          10019, 10020, 10036,10029, 10035, 10010, 10016, 10017, 
          10022,10012, 10013, 10014,10004, 10005, 10006, 10007, 
          10038, 10280, 10002, 10003, 10009,10021, 10028, 10044, 
          10065, 10075, 10128,10023, 10024, 10025, 10031, 10032, 10033, 10034, 10040]

bk_zip = list(map(str, bk_zip))
mn_zip = list(map(str, mn_zip))

#source: https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm

In [174]:
def get_zip(location): 
    zipcode = geolocator.reverse(location).raw['address']['postcode']
    return(zipcode)

In [175]:
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
df['zip'] = 0

In [179]:
df.columns

Index(['Bike ID', 'Birth Year', 'End Station ID', 'End Station Latitude',
       'End Station Longitude', 'End Station Name', 'Gender',
       'Start Station ID', 'Start Station Latitude', 'Start Station Longitude',
       'Start Station Name', 'Start Time', 'Stop Time', 'Trip Duration',
       'User Type', 'bikeid', 'birth year', 'end station id',
       'end station latitude', 'end station longitude', 'end station name',
       'gender', 'start station id', 'start station latitude',
       'start station longitude', 'start station name', 'starttime',
       'stoptime', 'tripduration', 'usertype', 'zip'],
      dtype='object')

In [177]:
df.zip = list(map(lambda x: search.by_coordinate(x[0], x[1], returns=1)[0]['Zipcode'], df.location))

OperationalError: no such column: nan

In [105]:
df.to_csv('data_backup.csv')

In [103]:
def get_mnbk(zipcode): 
    if zipcode in bk_zip: 
        return(1)
    if zipcode in mn_zip:
        return(0)
    else: 
        return(2)
df['mnbk'] = 0
df.mnbk = list(map(get_mnbk, df.zip))

In [107]:
df['age'] = 2017 - df['Birth Year']

In [113]:
np.unique(df.zip)

array(['07302', '07304', '07305', '07306', '07307', '07310', '07311'], dtype=object)

In [108]:
ks2 = scipy.stats.ks_2samp(df[df.mnbk == 1]['age'], df[df.mnbk == 0]['age'])
print (ks2)
# since the pvalue is low, we can reject the null hypothesis

ValueError: zero-size array to reduction operation maximum which has no identity

0        49.0
1        39.0
2        29.0
3        31.0
4        31.0
5        52.0
6        35.0
7        33.0
8        52.0
9        59.0
10       40.0
11       48.0
12       56.0
13       32.0
14       31.0
15       43.0
16       40.0
17       30.0
18       32.0
19        NaN
20        NaN
21       39.0
22       31.0
23       50.0
24       26.0
25       33.0
26       42.0
27       26.0
28       48.0
29       38.0
         ... 
12896    31.0
12897    35.0
12898    41.0
12899    40.0
12900    27.0
12901    42.0
12902    43.0
12903    41.0
12904    29.0
12905    34.0
12906    76.0
12907    45.0
12908    39.0
12909    29.0
12910    30.0
12911    31.0
12912    23.0
12913    35.0
12914    28.0
12915    36.0
12916    30.0
12917    28.0
12918     NaN
12919    41.0
12920    27.0
12921    37.0
12922    31.0
12923    29.0
12924    29.0
12925    30.0
Name: age, Length: 47075, dtype: float64

In [ ]:
# your code here!
# wrangle the data as needed
# please perform the Pearson's test 
# and tell me what you find in terms of NULL hypothesis
r2, pval2 = scipy.stats.pearsonr(np.sort(df.age*df.bk), np.sort(df.age*df.mn))
print(r2, pval2)
print('the correlation between the trip duration taking place at day time and night time is strong and statistically significant, hence fail to reject null hypothesis')

In [ ]:

# wrangle the data as needed
# please perform the Spearman's test and tell me what you find in terms of NULL hypothesisc
r22, pval22= scipy.stats.spearmanr(df.age*df.bk, df.age*df.mn)
print(r22, pval22)
print('The correlation between the two data set is weak (the two data set are statistically different) and we reject the null hypothesis since p < alpha')